In [ ]:
import os, subprocess

In [ ]:
yolov4_base_path = "/home/erbl/Git/RealTime-Object-Detection/darknet-yolo/yolov4-tiny-220607"

In [ ]:
os.makedirs(yolov4_base_path, exist_ok=True)

# 1. darknet 설치
https://webnautes.tistory.com/1482

## 1.1 github clone

In [ ]:
os.chdir(yolov4_base_path)

In [ ]:
!git clone https://github.com/AlexeyAB/darknet.git

## 1.2 Makefile 수정
```
# GPU, CUDNN, OPENCV, LIBSO 1로 하기
GPU=1 
CUDNN=1 
CUDNN_HALF=0 
OPENCV=1 
AVX=0 
OPENMP=0 
LIBSO=1

# ARCH 주석 변경 (GPU: GeForce RTX 2070)
ARCH= -gencode arch=compute_75,code=[sm_75,compute_75]

# NVCC 주석 변경
NVCC=/usr/local/cuda-10.2/bin/nvcc
```

## 1.3 MAKE

In [ ]:
darknet_path = f"{yolov4_base_path}/darknet"
print(darknet_path)

In [ ]:
os.chdir(darknet_path)
!pwd

In [ ]:
!make

---

# 2. darknet 파일 추가, 수정하기

폴더 구조
- darknet
  - data folder
    - labels folder
    - obj folder
    - obj.data file
    - obj.names file
    - (test.txt)
    - (train.txt)
  - cfg folder
    - yolov4-tiny-custom.cfg
  - yolov4-tiny.weights
- training

## 2.1. cfg 폴더의 yolov4-tiny-custom.cfg 수정

1. yolov4-tiny-custom.cfg 외 모든 파일 삭제
2. yolov4-tiny-custom.cfg 수정
   - batch=64
   - **subdivision=16 or 32 or 64**
   - width=416, height=416
   - max_batches = (class 개수) * 2000 (단, training images 수 이상, 6000 이상)
   - steps=80% and 90% of max_batches
   - [yolo] 직전에 있는, 3개의 [convolutional] filters=(classes + 5) * 3
   - 3개의 [yolo] classes = (class 개수)
   - **learning rate, angle, saturation, exposure, hue 추가로 바꿀 수 있음**

> yolov4-tiny-custom.cfg 수정 결과
>   - batch=64
>   - **subdivision=16 or 32 or 64** 
>     - 32 설정
>   - width=416, height=416
>   - max_batches = 8000
>     - Train Dataset(7762)
>   - steps = 6400,7200
>   - [yolo] 직전에 있는, 3개의 [convolutional] filters=21
>   - 3개의 [yolo] classes = 2
>   - **learning rate, angle, saturation, exposure, hue 추가로 바꿀 수 있음**

## 2.2. data 폴더
1. labels 폴더 외 모두 삭제
2. obj.data
3. obj.names

### 2.2.2. obj.data

In [ ]:
# 클래스 인데싱 딕셔너리
class_dir = {'closer': 0, 'stop': 1} # 수정, 추가 필요

In [ ]:
with open(darknet_path+"/data/obj.data", "w") as f:
    lines = f"classes = {len(class_dir)}\n"
    lines += f"train = {darknet_path}/data/train.txt\n"
    lines += f"valid = {darknet_path}/data/valid.txt\n"
    lines += f"names = {darknet_path}/data/obj.names\n"
    lines += f"backup = {yolov4_base_path}/training\n"
    f.write(lines)

### 2.2.3. obj.names

In [ ]:
with open(darknet_path+"/data/obj.names", "w") as f:
    lines = ""
    for name in class_dir.keys():
        lines += name + '\n'
    f.write(lines)

## 2.3. 학습 결과 weights 파일 저장될 Training 폴더 만들기

In [ ]:
os.makedirs(yolov4_base_path+"/training", exist_ok=True)

---

# 3. Labeled Custom Data

1. 로보플로우에서 데이터 다운 후, person 데이터셋 합치기
2. 데이터셋 파일 리스트 만들기

## 3.1. 로보플로우에서 데이터 다운
위치: /home/erbl/data/roboflow/

In [ ]:
data_base_path = "/home/erbl/data/roboflow/"

In [ ]:
os.chdir(data_base_path)
!pwd

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="X2gvBX7Me32fy3mU0Fkz")
project = rf.workspace("ebrlhfr-interaction").project("hfr_interaction")
dataset = project.version(2).download("darknet")

In [ ]:
data_path = f"{data_base_path}/HFR_Interaction-2"

## 3.2. 데이터셋 train.txt & test.txt 만들기

In [ ]:
import glob

def create_image_path_txt_file(dataset_type, data_path=data_path, darknet_path=darknet_path):
    image_list = glob.glob(f"{data_path}/{dataset_type}/*.jpg")
    with open(f"{darknet_path}/data/{dataset_type}.txt", "w") as f:
        f.write("\n".join(image_list) + "\n")


In [ ]:
create_image_path_txt_file("train")

In [ ]:
create_image_path_txt_file("valid")

---

# 4. download weights

In [ ]:
os.chdir(darknet_path)
!pwd

In [ ]:
download_weight = "wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights"
subprocess.call(list(download_weight.split()))

---
# 5. Training

In [ ]:
print(yolov4_base_path)
print(darknet_path)

In [ ]:
os.chdir(darknet_path)
!pwd

In [ ]:
train_yolo_tiny = f"./darknet detector train {darknet_path}/data/obj.data {darknet_path}/cfg/yolov4-tiny-custom.cfg {darknet_path}/yolov4-tiny.weights -map"

In [ ]:
subprocess.call(list(train_yolo_tiny.split()))

# Check mAP

darknet/cfg/yolov4-tiny-custom.cfg 파일 변경하고 TEST하기

In [ ]:
test_map_yolo_tiny = f"./darknet detector map {darknet_path}/data/obj.data {darknet_path}/cfg/yolov4-tiny-custom.cfg {yolov4_base_path}/training/yolov4-tiny-custom_best.weights -points 0"

In [ ]:
subprocess.call(list(test_map_yolo_tiny.split()))

# Run detector on a live webcam

In [ ]:
test_live_webcam_yolo_tiny = f"./darknet detector demo {darknet_path}/data/obj.data {darknet_path}/cfg/yolov4-tiny-custom.cfg {yolov4_base_path}/training/yolov4-tiny-custom_best.weights -thresh 0.5"

In [ ]:
subprocess.call(list(test_live_webcam_yolo_tiny.split()))